In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../qdmt/')

In [3]:
from uMPSHelpers import *
from optimise import genAContr

In [4]:
import numpy as np
from scipy.stats import unitary_group

In [5]:
def uniformToRhoN(A, N, l=None, r=None):
    '''
    Generate $\rho(A)_N$ which is the density matrix over $N$ sites.
    '''
    if l is None or r is None:
        l, r = fixedPoints(A)
    l, r = fixedPoints(A)

    tensors = [l, *[A]*N, *[A.conj()]*N, r]
    edges = [(2, 1)]
    i = 3
    edgesA = [(1, -1, i)]
    edgesAdag = [(2, -N-1, i+1)]
    i += 2
    for j in range(N-1):
        edgesA.append((i-2, -2-j, i))
        edgesAdag.append((i-1, -N-2-j, i+1))
        i += 2

    i = edgesA[-1][2]
    j = edgesAdag[-1][2]
    edges = edges + edgesA + edgesAdag + [(i, j)]
    return ncon(tensors, edges)


## Implement evolution 

In [6]:
# Initialise evolution unitary 
d = 2
D = 4
U = unitary_group.rvs(d**2)
U_tensor = U.reshape(d, d, d, d)

In [7]:
A = createMPS(D, d)
A = normalizeMPS(A)

In [8]:
N = 4
rhoA = uniformToRhoN(A, N=N)

In [9]:
rhoM = rhoA.reshape(d**N, d**N)

In [10]:
UU = ncon([U, U], ((-1, -3), (-2, -4))).reshape(d**N, d**N)

In [11]:
rhoMUU = UU @ rhoM @ UU.conj().T
rhoMUU = rhoMUU.reshape(*[d]*(N*2))

In [12]:
def evolve_rho_first_layer(A, U, N, l=None, r=None):
    if l is None or r is None: 
        l, r = fixedPoints(A)
        
    assert N % 2 == 0, 'N needs to be even'
    Acontr = genAContr(1, 3, 5, 4, N)
    AdagContr = genAContr(2, 4, 6, 4, N)
    # print('Adagcontr: ', AdagContr)
    
    botLeg1 = [a[1] for a in Acontr[::2]]
    botLeg2 = [a[1] for a in Acontr[1::2]]
    topLeg1 = range(-1, -1*N-1, -2)
    topLeg2 = range(-2, -1*N-1, -2)
    
    Ucontr = [list(a) for a in zip(topLeg1, topLeg2, botLeg1, botLeg2)]
    # print(Udagcontr)
    
    botLeg1 = [a[1] for a in AdagContr[::2]]
    botLeg2 = [a[1] for a in AdagContr[1::2]]
    topLeg1 = range(-7, -N-8, -2)
    topLeg2 = range(-8, -N-9, -2)
    
    # print(topLeg1)
    # print(list(topLeg1))
    # print(topLeg2)
    # print(list(topLeg2))
    Udagcontr = [list(a) for a in zip(topLeg1, topLeg2, botLeg1, botLeg2)]
    # print(Ucontr)
    
    tensors = [l] + [A]*N + [A.conj()]*N + [U.conj()]*(N//2) + [U]*(N//2) + [r]
    contr = [[AdagContr[0][0], Acontr[0][0]]] + Acontr + AdagContr + Udagcontr + Ucontr + [[Acontr[-1][-1], AdagContr[-1][-1]]] 
    print(contr)
    
    return ncon(tensors, contr)

In [13]:
evolvedRhoTensor = evolve_rho_first_layer(A, U_tensor, 4)

[[2, 1], [1, 3, 5], [5, 7, 9], [9, 11, 13], [13, 15, 17], [2, 4, 6], [6, 8, 10], [10, 12, 14], [14, 16, 18], [-7, -8, 4, 8], [-9, -10, 12, 16], [-1, -2, 3, 7], [-3, -4, 11, 15], [17, 18]]


In [14]:
tensors = [U_tensor, U_tensor, U_tensor.conj(), U_tensor.conj(), rhoA]
contr = [
    (-1, -2, 1, 2), (-3, -4, 3, 4), (-5, -6, 5, 6), (-7, -8, 7, 8), 
    (1, 2, 3, 4, 5, 6, 7, 8)
]
evolvedRhoUtensor = ncon(tensors, contr)

In [15]:
np.allclose(evolvedRhoTensor, evolvedRhoUtensor)

True

In [16]:
np.allclose(rhoMUU, evolvedRhoTensor)

True

In [17]:
np.allclose(rhoMUU, evolvedRhoUtensor)

True

In [18]:
testrhoMUU = ncon([UU, rhoM, UU.conj()], ((-1, 1), (1, 2), (-2, 2)))

In [19]:
np.allclose(rhoMUU.reshape(d**N, d**N), testrhoMUU)

True

## Apply even and odd unitaries

In [6]:
# Initialise evolution unitary 
d = 2
D = 4
U1 = unitary_group.rvs(d**2)
U1ten = U1.reshape(d, d, d, d)
U2 = unitary_group.rvs(d**2)
U2ten = U2.reshape(d, d, d, d)

A = createMPS(D, d)
A = normalizeMPS(A)
l, r = fixedPoints(A)
N = 6

In [7]:
# Acontr
AContr = genAContr(1, 4, 7, 6, N)
AContr

[[1, 4, 7],
 [7, 10, 13],
 [13, 16, 19],
 [19, 22, 25],
 [25, 28, 31],
 [31, 34, 37]]

In [8]:
# AdagContr
ADagContr = genAContr(2, 5, 8, 6, N)
ADagContr

[[2, 5, 8],
 [8, 11, 14],
 [14, 17, 20],
 [20, 23, 26],
 [26, 29, 32],
 [32, 35, 38]]

In [9]:
def genUContr(topLeft, topRight, botLeft, botRight, stepTop, stepBot, Nu=2):
    
    topLeft = range(topLeft, topLeft + stepTop*Nu, stepTop)
    topRight = range(topRight, topRight + stepTop*Nu, stepTop)
    botLeft = range(botLeft, botLeft + Nu*stepBot + 1, stepBot)
    botRight = range(botRight, botRight + Nu*stepBot + 1, stepBot)
    return [list(a) for a in zip(topLeft, topRight, botLeft, botRight)]

In [10]:
# Even layer U for A
evenAContr = genUContr(3, 9, 4, 10, 12, 12, 3)
#evenAContr[0][0] = -1
#evenAContr[-1][1] = -N
evenAContr

[[3, 9, 4, 10], [15, 21, 16, 22], [27, 33, 28, 34]]

In [11]:
# Odd layer U for A
oddAContr = genUContr(-1, -2, 9, 15, -2, 12, 2)
oddAContr

[[-1, -2, 9, 15], [-3, -4, 21, 27]]

In [12]:
# Even layer for Adag
evenADagContr = genUContr(6, 12, 5, 11, 12, 12, 3)
evenADagContr[0][0] = evenAContr[0][0]
evenADagContr[-1][1] = evenAContr[-1][1]
evenADagContr

[[3, 12, 5, 11], [18, 24, 17, 23], [30, 33, 29, 35]]

In [13]:
# Odd layer for Adag
oddADagContr = genUContr(-5, -6, 12, 18, -2, 12, 2)
oddADagContr

[[-5, -6, 12, 18], [-7, -8, 24, 30]]

In [14]:
# l/r contr
lcontr = [ADagContr[0][0], AContr[0][0]]
rcontr = [AContr[-1][2], ADagContr[-1][2]]

In [15]:
'''
print(lcontr)
print(rcontr)
print(AContr)
print(ADagContr)
print(evenAContr)
print(oddAContr)
print(evenADagContr)
print(oddADagContr)
'''

'\nprint(lcontr)\nprint(rcontr)\nprint(AContr)\nprint(ADagContr)\nprint(evenAContr)\nprint(oddAContr)\nprint(evenADagContr)\nprint(oddADagContr)\n'

In [16]:
rho6 = uniformToRhoN(A, N, l=l, r=r).reshape(d**N, d**N)
I = np.eye(2, dtype=complex)

In [17]:
UUU = ncon([U1ten, U1ten, U1ten], [(-1, -2, -7, -8), (-3, -4, -9, -10), (-5, -6, -11, -12)]).reshape(d**N, d**N)
IUUI = ncon([I, U2ten, U2ten, I], [(-1, -7), (-2, -3, -8, -9), (-4, -5, -10, -11), (-6, -12)]).reshape(d**N, d**N)

In [18]:
evolvedRho6 = IUUI @ UUU @ rho6 @ UUU.conj().T @ IUUI.conj().T
evolvedRho6 = evolvedRho6.reshape([d]*(2*N))
evolvedRho6 = ncon([evolvedRho6,], ((1, -1, -2, -3, -4, 2, 1, -5, -6, -7, -8, 2),))

In [19]:
tensors = [l, r] + [A]*N + [A.conj()]*N + [U1ten]*3 + [U2ten]*2 + [U1ten.conj()]*3 + [U2ten.conj()]*2 
contr = [lcontr, rcontr] + AContr + ADagContr + evenAContr + oddAContr + evenADagContr + oddADagContr

In [20]:
evolvedRho6Ten = ncon(tensors, contr)

In [21]:
print(contr)

[[2, 1], [37, 38], [1, 4, 7], [7, 10, 13], [13, 16, 19], [19, 22, 25], [25, 28, 31], [31, 34, 37], [2, 5, 8], [8, 11, 14], [14, 17, 20], [20, 23, 26], [26, 29, 32], [32, 35, 38], [3, 9, 4, 10], [15, 21, 16, 22], [27, 33, 28, 34], [-1, -2, 9, 15], [-3, -4, 21, 27], [3, 12, 5, 11], [18, 24, 17, 23], [30, 33, 29, 35], [-5, -6, 12, 18], [-7, -8, 24, 30]]


In [22]:
np.allclose(evolvedRho6, evolvedRho6Ten)

True

In [23]:
from evolve import firstOrderTrotterEvolve

In [24]:
evolvedCode = firstOrderTrotterEvolve(A, U1, U2, 4, l, r)

[[2, 1], [37, 38], [1, 4, 7], [7, 10, 13], [13, 16, 19], [19, 22, 25], [25, 28, 31], [31, 34, 37], [2, 5, 8], [8, 11, 14], [14, 17, 20], [20, 23, 26], [26, 29, 32], [32, 35, 38], [3, 9, 4, 10], [15, 21, 16, 22], [27, 33, 28, 34], [-1, -2, 9, 15], [-3, -4, 21, 27], [3, 12, 5, 11], [18, 24, 17, 23], [30, 33, 29, 35], [-5, -6, 12, 18], [-7, -8, 24, 30]]


In [25]:
evolvedCode.shape

(2, 2, 2, 2, 2, 2, 2, 2)

In [26]:
np.linalg.norm(evolvedCode - evolvedRho6)

5.480779506258892e-16

In [35]:
np.allclose(evolvedRho6, evolvedCode)

True

In [36]:
np.allclose(evolvedRho6Ten, evolvedCode)

True

In [35]:
def testFirstOrderTrotterEvolve():
    d = 2
    D = 4
    U1 = unitary_group.rvs(d**2)
    U1ten = U1.reshape(d, d, d, d)
    U2 = unitary_group.rvs(d**2)
    U2ten = U2.reshape(d, d, d, d)
    I = np.eye(2, dtype=complex)

    A = createMPS(D, d)
    A = normalizeMPS(A)

    l, r = fixedPoints(A)

    # Check 4 site case
    N = 6 # Need extra sites on either side
    rho6 = uniformToRhoN(A, N, l=l, r=r).reshape(d**N, d**N)

    UUU = ncon([U1ten, U1ten, U1ten], [(-1, -2, -7, -8), (-3, -4, -9, -10), (-5, -6, -11, -12)]).reshape(d**N, d**N)
    IUUI = ncon([I, U2ten, U2ten, I], [(-1, -7), (-2, -3, -8, -9), (-4, -5, -10, -11), (-6, -12)]).reshape(d**N, d**N)

    evolvedRho6 = IUUI @ UUU @ rho6 @ UUU.conj().T @ IUUI.conj().T

    evolvedRho6 = evolvedRho6.reshape([d]*(2*N))
    evolvedRho6 = ncon([evolvedRho6,], ((1, -1, -2, -3, -4, 2, 1, -5, -6, -7, -8, 2),))

    evolvedTen = firstOrderTrotterEvolve(A, U1, U2, 4, l=l, r=r)

    print(np.allclose(evolvedRho6, evolvedTen))
    print(np.linalg.norm(evolvedTen - evolvedRho6))

In [36]:
testFirstOrderTrotterEvolve()

24
[[2, 1], [37, 38], [1, 4, 7], [7, 10, 13], [13, 16, 19], [19, 22, 25], [25, 28, 31], [31, 34, 37], [2, 5, 8], [8, 11, 14], [14, 17, 20], [20, 23, 26], [26, 29, 32], [32, 35, 38], [3, 9, 4, 10], [15, 21, 16, 22], [27, 33, 28, 34], [-1, -2, 9, 15], [-3, -4, 21, 27], [3, 12, 5, 11], [18, 24, 17, 23], [30, 33, 29, 35], [-5, -6, 12, 18], [-7, -8, 24, 30]]
True
3.7190833695294273e-16


In [40]:
trList = list(range(N))
print(trList)
trList = trList[N//2:] + trList[:N//2]
print(trList)

[0, 1, 2, 3, 4, 5]
[3, 4, 5, 0, 1, 2]
